In [2]:
#from Loader import fileloader_proto as fl
#from Loader import dfutils
import ecomis
import xlwings as xw
import pandas as pd

month = '202005'

inifile = ecomis.ReadIniFile(mercado="empresas")

In [2]:
#The final order the columns written in the excel file will have
columns_resultant = ['DNI', 'PADRON_FECHA_DE_INGRESO', 'FECHA_DE_INICIO_EN_EL_PUESTO', 'NOMBRE_DEL_PUESTO', 
                     'FECHA_CESE','FECHA_INICIO_SUBSIDIO','FECHA_FIN_SUBSIDIO','TARGET']

#Generando dataframes de empleados, ceses y subsidios
#Padron Empleados
section_padron_empleados = ecomis.SectionObj(inifile,'Padron_Empleados',month)
loader_empleados = ecomis.LoadFileProcess(section_padron_empleados)
padron_df = loader_empleados.loadFile()
padron_df_short = padron_df[ columns_resultant[0:4] ]

#Ceses
section_ceses = ecomis.SectionObj(inifile,'Ceses',month)
loader_ceses = ecomis.LoadFileProcess(section_ceses)
ceses_df = loader_ceses.loadFile()
ceses_df_short = ceses_df[[ columns_resultant[0], columns_resultant[4] ]]
#Subsidios
section_subsidios = ecomis.SectionObj(inifile,'Subsidios',month)
loader_subsidios = ecomis.LoadFileProcess(section_subsidios)
subsidios_df = loader_subsidios.loadFile()

#Targets
section_targets = ecomis.SectionObj(inifile,'Targets',month)
loader_targets = ecomis.LoadFileProcess(section_targets)
targets_df = loader_targets.loadFile()





In [7]:
# Iteration over the comisiones Excel workbooks
"""
inis = ['Comisionantes_Plataformas_All', 'Comisionantes_CAL_All', 'Comisionantes_Fidelizacion_Telemarketing_All', 
'Comisionantes_Soporte_Empresas_All', 'Comisionantes_Corporaciones_All', 'Comisionantes_SolucionesNegocio_All', 
'Comisionantes_Pymes_All','Comisionantes_GC_DDNN_IS_All','Comisionantes_VentaRegionaEmpresa_All']
"""

inis = [ 'Comisionantes_Pymes_All']

for i in range(len(inis)):
  # Generating the dataframes.
  section_ingresos = ecomis.SectionObj(inifile,inis[i],month)
  loader = ecomis.LoadFileProcess(section_ingresos)
  comisionantes_df = loader.loadFile()

  col_index = ecomis.getExcelColIndexFromDF(comisionantes_df, 'DNI2')
  row_len = len(comisionantes_df.index)

  comis_skiprows = section_ingresos.getParameter('skiprows')
  file = section_ingresos.getParameter('filelist')[0]

  #Tomamnos solo la columna DNI, este sería nuestra referencia para concatenar/merge la demas data a la izquierda.
  comisionantes_df_short = comisionantes_df[[columns_resultant[0]]].copy()

  # Merging the dataframes
  merged_1 = pd.merge(comisionantes_df_short, padron_df_short, how='left',left_on='DNI', right_on = 'DNI')
  merged_2 = pd.merge(merged_1, ceses_df_short, how='left',left_on='DNI', right_on = 'DNI')
  merged_3 = pd.merge(merged_2, subsidios_df, how='left',left_on='DNI', right_on = 'DNI')
  merged_4 = pd.merge(merged_3, targets_df, how='left',left_on='DNI', right_on = 'DNI')
  merged_4.drop_duplicates('DNI', inplace = True)
  # Rename DNI -> DNI2 because the column name already exists in the sheet
  merged_4.rename(columns={'DNI':'DNI2'},inplace=True)

  # Opening the workbook and referencing to 'comisionantes' worksheet
  wb = xw.Book(file)
  comis_sheet = wb.sheets('Comisionantes')

  # Pasting the resultant dataframe of the merge operations  to the Excel sheet
  comis_sheet.range((comis_skiprows + 1,col_index),(row_len + comis_skiprows + 1,col_index)).number_format = '@'
  comis_sheet.range((comis_skiprows + 1,col_index)).options(pd.DataFrame, index=False).value = merged_4

  wb.save()
  wb.close()